In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

import pickle
import numpy as np

f2 = open('data/split_0.80_0.10_0.10_aug_to_2500_down_sample_True_instance0-9/val_data', 'rb')
X_sequence_val, X_feature_val, X_image_val, Y_val = pickle.load(f2)
f2.close()

from keras.preprocessing.sequence import pad_sequences
from util import preprocess

total_X_sequence_train = []
total_X_feature_train = []
# total_X_image_train = []
total_Y_train = []
for i in range(0,10):
    f = open('data/split_0.80_0.10_0.10_aug_to_2500_down_sample_True_instance0-9/train_data%d'%i, 'rb')
    X_sequence_train, X_feature_train, X_image_train, Y_train = pickle.load(f)
    X_sequence_train_paded = pad_sequences(X_sequence_train, dtype='float', padding='post', value = np.nan)
    X_sequence_train_trimed = preprocess(X_sequence_train_paded)
    X_feature_train = np.array(X_feature_train)
    Y_train = np.array(Y_train)-1
    total_X_sequence_train.append(X_sequence_train_trimed)
    total_X_feature_train.append(X_feature_train)
    total_Y_train.append(Y_train)
    f.close()

X_sequence_val_paded = pad_sequences(X_sequence_val, dtype='float', padding='post', value = np.nan)
X_sequence_val_trimed = preprocess(X_sequence_val_paded)
X_feature_val = np.array(X_feature_val)
Y_val = np.array(Y_val)-1


In [2]:
def rnn_with_feature(sequence_shape, feature_shape, num_classes):
    sequence_inputs = keras.Input(shape=sequence_shape, name='sequence')
    masking_layer = layers.Masking(mask_value=0)
    x1 = masking_layer(sequence_inputs)
    x1 = layers.Bidirectional(layers.GRU(128, return_sequences=True))(x1)
    x1 = layers.Bidirectional(layers.GRU(64))(x1)
    x1 = layers.Dropout(0.5)(x1)
    x1 = layers.Dense(128, activation="relu")(x1)
    x1 = layers.Dropout(0.5)(x1)
    x1 = layers.Dense(32, activation="relu")(x1)

    feature_inputs = keras.Input(shape=feature_shape, name='feature')
    x3 = layers.Dense(32, activation="relu")(feature_inputs)
    x3 = layers.Dense(16, activation="relu")(x3)

    x = layers.concatenate([x1, x3])
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = keras.Model(inputs=[sequence_inputs, feature_inputs], outputs=outputs)
    return model

In [5]:
X_val = {'sequence':X_sequence_val_trimed, 'feature':X_feature_val}
for i in range(1,10):

    model = rnn_with_feature(sequence_shape=(None, 3),feature_shape=(2,), num_classes=11)
    model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    run_eagerly=False,
    )
    callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='models/bagging/rnn_with_feature%d/model_{epoch}'%i,
        save_freq='epoch'),
    keras.callbacks.TensorBoard(log_dir='models/bagging/rnn_with_feature%d/logs'%i)
    ]
    X_train = {'sequence':total_X_sequence_train[i], 'feature':total_X_feature_train[i]}
    model.fit(X_train, total_Y_train[i], batch_size=32, epochs=10, 
        validation_data=(X_val, Y_val), callbacks=callbacks)


2022-10-13 13:16:51.067324: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-10-13 13:16:51.067364: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-10-13 13:16:51.696479: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-10-13 13:16:51.696710: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed


Epoch 1/10
  1/860 [..............................] - ETA: 2:04:18 - loss: 3.6146 - acc: 0.0938

2022-10-13 13:17:00.702729: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-10-13 13:17:00.702772: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


  2/860 [..............................] - ETA: 10:57 - loss: 4.8813 - acc: 0.0781  

2022-10-13 13:17:01.381491: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-10-13 13:17:01.381848: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
2022-10-13 13:17:01.434081: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 5653 callback api events and 5650 activity events. 
2022-10-13 13:17:01.515037: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-10-13 13:17:01.613059: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: models/bagging/rnn_with_feature1/logs/train/plugins/profile/2022_10_13_13_17_01

2022-10-13 13:17:01.684996: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to models/bagging/rnn_with_feature1/logs/train/plugins/profile/2022_10_13_13_17_01/dell-Precision-7920-Tower.trace.json.gz
2022-10-13 13:17:01.745784: I tensorflow/

...

INFO:tensorflow:Assets written to: models/bagging/rnn_with_feature9/model_9/assets


INFO:tensorflow:Assets written to: models/bagging/rnn_with_feature9/model_9/assets


Epoch 10/10
860/860 [==============================] - 82s 96ms/step - loss: 0.2853 - acc: 0.9047 - val_loss: 0.7674 - val_acc: 0.8170


INFO:tensorflow:Assets written to: models/bagging/rnn_with_feature9/model_10/assets


INFO:tensorflow:Assets written to: models/bagging/rnn_with_feature9/model_10/assets
